In [ ]:
from arcgis.gis import GIS
import ast
import json
import json5
import pathlib
import requests

cwd = pathlib.Path().cwd()
def_dir = cwd.parent / 'layer_definitions' / '20250317'
proj_dir = cwd.parent.parent
secrets_dir = proj_dir / 'secrets'

In [ ]:
## retrieve token

with open(secrets_dir / 'nifc_ago_credentials.txt', 'r') as file:
    nifc_credentials_str = file.read()
    nifc_credentials_tup = ast.literal_eval(nifc_credentials_str)

nifc_gis = GIS(*nifc_credentials_tup)

token = nifc_gis._con.token

In [ ]:
# for some reason the order in which layers are added is dictating their url endpoint id - not the "id": # parameter in the definition
# going through and doing them manually to ensure the correct order is maintained
for entry in def_dir.iterdir():

    if entry.suffix == '.json5':
        print(entry)

c:\REPOS\con-j-e\ak-wildfire-values-at-risk\feature_service_management\layer_definitions\20250317\Fire_Perimeters_and_Reported_Locations.json5
c:\REPOS\con-j-e\ak-wildfire-values-at-risk\feature_service_management\layer_definitions\20250317\Five_Mile_Fire_Buffers.json5
c:\REPOS\con-j-e\ak-wildfire-values-at-risk\feature_service_management\layer_definitions\20250317\One_Mile_Fire_Buffers.json5
c:\REPOS\con-j-e\ak-wildfire-values-at-risk\feature_service_management\layer_definitions\20250317\Three_Mile_Fire_Buffers.json5

In [ ]:

# manually repoint this to each path printed above, going in desired order for the feature service.
entry = pathlib.Path(r'c:\REPOS\con-j-e\ak-wildfire-values-at-risk\feature_service_management\layer_definitions\20250317\Five_Mile_Fire_Buffers.json5')

admin_url = r'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/admin/services/AK_Wildfire_Values_at_Risk/FeatureServer'
#-admin_url = r'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/admin/services/TESTING_AK_Wildfire_Values_at_Risk/FeatureServer'

with open(entry, 'r') as json5_file:
    layer_definition = json5.load(json5_file)

# saving json5 as json before posting data
with open(def_dir / f'{entry.stem}.json', 'w') as json_file:
    json.dump(layer_definition, json_file, indent=4)

with open(def_dir / f'{entry.stem}.json', 'r') as json_file:
    layer_definition = json.load(json_file)

payload = {
    'addToDefinition': json.dumps(layer_definition),
    'f': 'json',
    'token': token
}

response = requests.post(
    fr'{admin_url}/addToDefinition',
    data=payload
)

print(response.json())